In [18]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity

In [20]:
interactions_df = pd.read_csv('interactions_processed_kion.csv')
users_df = pd.read_csv('users_processed_kion.csv')
items_df = pd.read_csv('items_processed_kion.csv')

In [21]:
interactions_df['t_dat'] = pd.to_datetime(interactions_df['last_watch_dt'], format="%Y-%m-%d")
interactions_df['timestamp'] = interactions_df.t_dat.values.astype(np.int64) // 10 ** 9

In [22]:
df = interactions_df[['user_id', 'item_id', 'timestamp']].rename(
    columns={'user_id': 'user_id:token', 'item_id': 'item_id:token', 'timestamp': 'timestamp:float'})

In [23]:
df

,user_id:token,item_id:token,timestamp:float
0,176549,9506,1620691200
1,699317,1659,1622246400
2,656683,7107,1620518400
3,864613,7638,1625443200
4,964868,9506,1619740800
...,...,...,...
5476246,648596,12225,1628812800
5476247,546862,9673,1618272000
5476248,697262,15297,1629417600
5476249,384202,16197,1618790400


In [25]:
df.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

In [28]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec, Caser
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.quick_start import run_recbole

In [29]:
parameter_dict = {
    'data_path': '',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'device': 'GPU',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 10,
    'verbose': -1,
    'show_progress' : False,
    'eval_args': {
        'split': {'RS': [9, 0, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}
config = Config(model='MultiVAE', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
# logger.info(config)

In [30]:
dataset = create_dataset(config)
logger.info(dataset)

11 Dec 11:56    INFO  recbox_data
The number of users: 13355
Average actions of users: 63.815710648494836
The number of items: 3294
Average actions of items: 258.78985727300335
The number of inters: 852195
The sparsity of the dataset: 98.06281322904924%
Remain Fields: ['user_id', 'item_id', 'timestamp']
recbox_data
The number of users: 13355
Average actions of users: 63.815710648494836
The number of items: 3294
Average actions of items: 258.78985727300335
The number of inters: 852195
The sparsity of the dataset: 98.06281322904924%
Remain Fields: ['user_id', 'item_id', 'timestamp']


In [31]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

11 Dec 11:56    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
11 Dec 11:56    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}]


In [32]:
import time

### Использование различных архитектур

In [33]:
%%time
model_list = [ "LightGCN", "MultiVAE", "RecVAE"] 

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name, dataset = 'recbox_data',config_dict = parameter_dict)
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)

running LightGCN...


11 Dec 11:56    INFO  ['/Users/annapikuleva/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py', '--f=/Users/annapikuleva/Library/Jupyter/runtime/kernel-v2-3832937JAU6uqtVOE.json']
['/Users/annapikuleva/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py', '--f=/Users/annapikuleva/Library/Jupyter/runtime/kernel-v2-3832937JAU6uqtVOE.json']
11 Dec 11:56    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4



It took 21.95 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0792), ('mrr@10', 0.1685), ('ndcg@10', 0.0795), ('hit@10', 0.3385), ('precision@10', 0.0441)])}
running MultiVAE...


11 Dec 12:18    INFO  ['/Users/annapikuleva/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py', '--f=/Users/annapikuleva/Library/Jupyter/runtime/kernel-v2-3832937JAU6uqtVOE.json']
['/Users/annapikuleva/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py', '--f=/Users/annapikuleva/Library/Jupyter/runtime/kernel-v2-3832937JAU6uqtVOE.json']
11 Dec 12:18    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4



It took 3.87 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0839), ('mrr@10', 0.1687), ('ndcg@10', 0.0823), ('hit@10', 0.3494), ('precision@10', 0.0465)])}
running RecVAE...


11 Dec 12:22    INFO  ['/Users/annapikuleva/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py', '--f=/Users/annapikuleva/Library/Jupyter/runtime/kernel-v2-3832937JAU6uqtVOE.json']
['/Users/annapikuleva/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py', '--f=/Users/annapikuleva/Library/Jupyter/runtime/kernel-v2-3832937JAU6uqtVOE.json']
11 Dec 12:22    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4



It took 6.70 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0844), ('mrr@10', 0.1662), ('ndcg@10', 0.0816), ('hit@10', 0.3519), ('precision@10', 0.0468)])}
CPU times: user 25min 39s, sys: 9min 10s, total: 34min 49s
Wall time: 32min 31s


In [35]:
%pip install kmeans-pytorch

Note: you may need to restart the kernel to use updated packages.


In [36]:
from kmeans_pytorch import kmeans

In [37]:
%%time
model_list = ["CORE", "LightSANs", "NextItNet",] 

parameter_dict["train_neg_sample_args"] = None

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name, dataset = 'recbox_data', config_dict = parameter_dict)
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)

running CORE...


11 Dec 13:40    INFO  ['/Users/annapikuleva/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py', '--f=/Users/annapikuleva/Library/Jupyter/runtime/kernel-v2-3832937JAU6uqtVOE.json']
['/Users/annapikuleva/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py', '--f=/Users/annapikuleva/Library/Jupyter/runtime/kernel-v2-3832937JAU6uqtVOE.json']
11 Dec 13:40    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place 

It took 463.62 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0921), ('mrr@10', 0.0297), ('ndcg@10', 0.044), ('hit@10', 0.0921), ('precision@10', 0.0092)])}
running LightSANs...


11 Dec 21:23    INFO  ['/Users/annapikuleva/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py', '--f=/Users/annapikuleva/Library/Jupyter/runtime/kernel-v2-3832937JAU6uqtVOE.json']
['/Users/annapikuleva/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py', '--f=/Users/annapikuleva/Library/Jupyter/runtime/kernel-v2-3832937JAU6uqtVOE.json']
11 Dec 21:23    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place 

It took 677.87 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.1029), ('mrr@10', 0.0358), ('ndcg@10', 0.0513), ('hit@10', 0.1029), ('precision@10', 0.0103)])}
running NextItNet...


12 Dec 08:41    INFO  ['/Users/annapikuleva/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py', '--f=/Users/annapikuleva/Library/Jupyter/runtime/kernel-v2-3832937JAU6uqtVOE.json']
['/Users/annapikuleva/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py', '--f=/Users/annapikuleva/Library/Jupyter/runtime/kernel-v2-3832937JAU6uqtVOE.json']
12 Dec 08:41    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place 

It took 814.55 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0922), ('mrr@10', 0.0329), ('ndcg@10', 0.0466), ('hit@10', 0.0922), ('precision@10', 0.0092)])}
CPU times: user 1d 24min 58s, sys: 13h 28min 5s, total: 1d 13h 53min 4s
Wall time: 1d 8h 36min 2s
